# 🚀 Projeto: Construção da Tabela Gold - Dimensão Cliente

## 📌 Visão Geral

Este módulo representa a construção da **tabela de dimensão `dim_cliente`** na camada **Gold** do projeto de Data Lakehouse. A tabela é derivada da Silver e foi desenhada para armazenar as informações consolidadas dos clientes, com foco em **qualidade dos dados**, **desempenho analítico** e **governança eficiente**.

---

## 🎯 Objetivo

Criar uma tabela dimensional que represente os clientes, garantindo:

- 🙋‍♂️ Dados únicos e limpos por cliente (`CustomerID`)
- ⚙️ Estrutura otimizada para consultas analíticas
- 🕒 Rastreabilidade com controle de criação (`Created_at`)
- 🔄 Atualizações eficientes via **MERGE Delta**
- 🚀 Alto desempenho com **Z-ORDER** e **reparticionamento estratégico**

---

## 📐 Detalhes Técnicos

- **Fonte**: Camada Silver (`tabela_prata_desnormalizadas`)
- **Destino**: Camada Gold (`gold.dim_cliente`)
- **Particionamento**: `Country`, `State` (4 partições)
- **Chave de Negócio**: `CustomerID`
- **Colunas da Dimensão Cliente**:
  - `CustomerID` (identificador do cliente)
  - `Name` (nome do cliente)
  - `PhoneNumber` (telefone)
  - `Age` (idade)
  - `City` (cidade)
  - `State` (estado)
  - `Country` (país)
  - `Created_at` (timestamp de inserção na tabela Gold)

---

## ⚙️ Etapas do Pipeline

1. **Criação do Banco de Dados `gold`**, caso não exista.
2. **Leitura da Tabela Silver** com dados brutos e normalizados.
3. **Reparticionamento Estratégico** com base em `Country` e `State`.
4. **Limpeza de Dados**: remoção de duplicatas e registros nulos.
5. **Cache dos Dados** para otimização de múltiplas operações.
6. **Adição de Auditoria Temporal** com a coluna `Created_at`.
7. **Auditoria Inicial**: contagem dos clientes únicos existentes.
8. **Aplicação do MERGE Delta**: inserção e atualização baseada em `CustomerID`.
9. **Auditoria Final**: nova contagem dos clientes únicos pós-processamento.
10. **Otimização com Z-ORDER** por `CustomerID` e `Country`.
11. **Limpeza com VACUUM** para liberação de espaço.
12. **Registro no Catálogo Hive/Unity Catalog** para consultas SQL.

---

## ✅ Benefícios Técnicos Aplicados

| Técnica                      | Finalidade                                                                |
|-----------------------------|---------------------------------------------------------------------------|
| `MERGE` Delta               | Evita duplicidade e garante atualizações incrementais                    |
| Reparticionamento estratégico | Aumenta performance de leitura por filtros geográficos (`Country`, `State`) |
| `dropDuplicates` + `na.drop` | Garante consistência e integridade dos dados                             |
| `Z-ORDER`                   | Melhora tempo de resposta em consultas filtradas por `CustomerID`         |
| `VACUUM`                    | Reduz uso de armazenamento eliminando arquivos obsoletos                 |
| Registro no catálogo        | Permite uso via SQL e dashboards com governança centralizada             |

---

## 🧱 Modelo Estrela (Star Schema)

Esta tabela é utilizada como uma **Dimensão** em um **Esquema Estrela**, onde será relacionada à Tabela Fato de Vendas através da chave `CustomerID`, permitindo análises como:

- Total de vendas por cliente
- Perfis demográficos dos consumidores (idade, localização)
- Foco regional em campanhas de marketing

---


In [0]:
from pyspark.sql.functions import col, current_timestamp, lit
from delta.tables import DeltaTable

# Caminho para as tabelas Delta
SILVER_PATH = "abfss://silver@dlsprojetofixo.dfs.core.windows.net/tabela_prata_desnormalizadas"
GOLD_DIM_PATH = "abfss://gold@dlsprojetofixo.dfs.core.windows.net/gold_dim_cliente"
GOLD_DIM_TABLE = "gold.dim_cliente"

# ------------------------------------------
# 1. Verifica se o banco de dados 'gold' existe e cria caso não exista
# ------------------------------------------
# Criar o banco de dados 'gold' se não existir para garantir que a tabela seja registrada corretamente.
spark.sql("CREATE DATABASE IF NOT EXISTS gold")

# ------------------------------------------
# 2. Carregar os dados da camada Silver
# ------------------------------------------
# Leitura dos dados da camada Silver, onde as informações brutas são armazenadas.
# Aqui, os dados são carregados de uma tabela Delta.
clientes_df = spark.read.format("delta").load(SILVER_PATH)

# ------------------------------------------
# 3. Reparticionamento estratégico
# ------------------------------------------
# Reparticionamento baseado em "Country" e "State" para melhorar a performance durante filtros geográficos.
# O reparticionamento deve ser feito para reduzir o tempo de leitura em operações futuras, especialmente em consultas 
# que fazem filtros por essas colunas. Usamos 4 partições como exemplo.
clientes_df = clientes_df.repartition(4, "Country", "State")

# ------------------------------------------
# 4. Limpeza robusta dos dados
# ------------------------------------------
# A limpeza de dados é um passo fundamental. 
# 1. Removemos duplicatas com base no campo 'CustomerID', que é uma chave única.
# 2. Eliminamos registros com valores nulos nas colunas essenciais (Name, PhoneNumber e Age).
clientes_df = clientes_df.dropDuplicates(["CustomerID"])
clientes_df = clientes_df.na.drop(subset=["Name", "PhoneNumber", "Age"])

# ------------------------------------------
# 5. Cache do DataFrame para otimizar múltiplas operações subsequentes
# ------------------------------------------
# O cache é utilizado aqui porque o DataFrame será utilizado em várias transformações, 
# o que pode levar a um ganho significativo de desempenho ao evitar recalcular o DataFrame repetidamente.
clientes_df.cache()

# ------------------------------------------
# 6. Adicionar uma coluna de data de criação
# ------------------------------------------
# Adiciona uma coluna 'Created_at' com a data e hora atual para rastrear quando os dados foram carregados ou atualizados.
clientes_df = clientes_df.withColumn("Created_at", current_timestamp())

# ------------------------------------------
# 7. Habilita a atualização automática de schema no Delta Lake
# ------------------------------------------
# Permite que o Delta aceite novos campos durante a escrita sem erro de schema incompatível.
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

# ------------------------------------------
# 8. MERGE - Atualizar ou inserir dados na tabela Delta Gold
# ------------------------------------------
# Usamos o MERGE para garantir que a tabela Delta seja atualizada com dados novos ou modificados.
# O MERGE realiza um "upsert", ou seja, ele faz update quando os registros existem e insere quando não existem.
# Verificamos se a tabela já existe. Caso exista, fazemos o merge, caso contrário, criamos a tabela do zero.
if DeltaTable.isDeltaTable(spark, GOLD_DIM_PATH):
    cliente_table = DeltaTable.forPath(spark, GOLD_DIM_PATH)

    # Realiza o MERGE: 
    cliente_table.alias("target").merge(
        clientes_df.alias("source"),
        "target.CustomerID = source.CustomerID"
    ).whenMatchedUpdate(set={
        "Name": col("source.Name"),
        "PhoneNumber": col("source.PhoneNumber"),
        "Age": col("source.Age"),
        "City": col("source.City"),
        "State": col("source.State"),
        "Country": col("source.Country"),
        "Created_at": col("source.Created_at")
    }).whenNotMatchedInsert(values={
        "CustomerID": col("source.CustomerID"),
        "Name": col("source.Name"),
        "PhoneNumber": col("source.PhoneNumber"),
        "Age": col("source.Age"),
        "City": col("source.City"),
        "State": col("source.State"),
        "Country": col("source.Country"),
        "Created_at": col("source.Created_at")
    }).execute()
    print("Tabela Gold Dimensão Cliente atualizada com sucesso.")
else:
    # Se a tabela não existir, criamos a tabela a partir dos dados carregados.
    clientes_df.write.format("delta").mode("append").option("mergeSchema", "true").save(GOLD_DIM_PATH)
    print("Tabela Gold Dimensão Cliente criada com sucesso.")

# ------------------------------------------
# 9. Z-ORDER para otimizar leitura por colunas frequentemente filtradas
# ------------------------------------------
# O Z-ORDER otimiza fisicamente os dados no Delta Lake para melhorar a performance de leitura,
# principalmente quando os dados são filtrados com frequência por determinadas colunas, como "CustomerID" e "Country".
# Isso organiza os dados em disco para facilitar a leitura rápida por essas colunas.
spark.sql(f"OPTIMIZE delta.`{GOLD_DIM_PATH}` ZORDER BY (CustomerID, Country)")
print("Tabela Gold Dimensão Cliente otimizada com sucesso.")

# ------------------------------------------
# 10. VACUUM para economia de armazenamento
# ------------------------------------------
# O comando VACUUM remove arquivos não referenciados no Delta Log que podem estar ocupando espaço desnecessário.
# A retenção de 168 horas (7 dias) garante que os dados sejam mantidos por um tempo suficiente antes de serem removidos.
spark.sql(f"VACUUM delta.`{GOLD_DIM_PATH}` RETAIN 168 HOURS")
print("Processo de VACUUM concluído, arquivos obsoletos removidos.")

# ------------------------------------------
# 11. Registro no catálogo para acesso via SQL e notebooks
# ------------------------------------------
# Registra a tabela no catálogo para facilitar o acesso em outros processos, notebooks ou consultas SQL.
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {GOLD_DIM_TABLE}
USING DELTA
LOCATION '{GOLD_DIM_PATH}'
""")
print("Tabela Gold Dimensão Cliente registrada no catálogo com sucesso.")

# ------------------------------------------
# 12. Validação e auditoria da carga
# ------------------------------------------
# Realiza a auditoria imprimindo o total de registros antes e depois do processamento
cliente_count_before = spark.read.format("delta").load(GOLD_DIM_PATH).select("CustomerID").distinct().count()
print(f"Total de registros distintos antes do processamento (Gold Dimensão Cliente): {cliente_count_before}")

cliente_count_after = spark.read.format("delta").load(GOLD_DIM_PATH).select("CustomerID").distinct().count()
print(f"Total de registros distintos após o processamento (Gold Dimensão Cliente): {cliente_count_after}")


Tabela Gold Dimensão Cliente atualizada com sucesso.
Tabela Gold Dimensão Cliente otimizada com sucesso.
Processo de VACUUM concluído, arquivos obsoletos removidos.
Tabela Gold Dimensão Cliente registrada no catálogo com sucesso.
Total de registros distintos antes do processamento (Gold Dimensão Cliente): 500
Total de registros distintos após o processamento (Gold Dimensão Cliente): 500


In [0]:
display(spark.read.table(GOLD_DIM_TABLE).dropDuplicates(["CustomerID"]))

CustomerID,Name,PhoneNumber,Age,City,State,Country,Created_at,data_atualizacao
1,Peter Arroyo,+1-510-4943,35,Kathrynfort,AK,Netherlands Antilles,2025-04-23T00:15:07.27Z,null
2,Noah Kramer,+1-322-2452,36,Danielsville,WV,Papua New Guinea,2025-04-23T00:15:07.27Z,null
3,Kirk Becker,+1-554-6529,69,Ryanchester,LA,France,2025-04-23T00:15:07.27Z,null
4,Jacqueline Gray,+1-700-4444,58,Jennifershire,AK,Congo,2025-04-23T00:15:07.27Z,null
5,Julie Hobbs,+1-157-9744,51,Patriciaborough,TX,American Samoa,2025-04-23T00:15:07.27Z,null
6,Philip Thompson,+1-843-5051,47,New Emilyside,AL,Moldova,2025-04-23T00:15:07.27Z,null
7,Lisa Becker,+1-381-5680,41,West Jessicaview,ND,Bermuda,2025-04-23T00:15:07.27Z,null
8,Marie Moon,+1-917-7575,41,Jacksonview,WI,Malawi,2025-04-23T00:15:07.27Z,null
9,Michael Johnson,+1-400-5224,48,Aaronberg,MI,Angola,2025-04-23T00:15:07.27Z,null
10,Victor Williams,+1-188-8635,70,Meyerhaven,WA,Estonia,2025-04-23T00:15:07.27Z,null


In [0]:
# 📊 Monitoramento e Governança - Log de Execução do Pipeline Gold (Dimensão Cliente)

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from datetime import datetime
import time

# ------------------------------------------
# 1. Início da contagem de tempo de execução
# ------------------------------------------
start_time = time.time()

# ------------------------------------------
# 2. Parâmetros do log
# ------------------------------------------
job_name = "gold_dim_cliente"
status = "SUCESSO"
erro = None

try:
    # Cálculo da quantidade de registros processados
    qtd_linhas = clientes_df.select("CustomerID").distinct().count()

except Exception as e:
    status = "ERRO"
    erro = str(e)
    qtd_linhas = 0

# ------------------------------------------
# 3. Cálculo do tempo total de execução (em segundos)
# ------------------------------------------
tempo_total = round(time.time() - start_time, 2)

# ------------------------------------------
# 4. Definição do schema do log
# ------------------------------------------
schema_log = StructType([
    StructField("job_name", StringType(), True),
    StructField("data_execucao", StringType(), True),
    StructField("qtd_linhas", IntegerType(), True),
    StructField("status", StringType(), True),
    StructField("erro", StringType(), True),
    StructField("tempo_total_segundos", DoubleType(), True)
])

# ------------------------------------------
# 5. Criação do DataFrame de log
# ------------------------------------------
log_execucao_df = spark.createDataFrame([(
    job_name,
    datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    qtd_linhas,
    status,
    erro,
    tempo_total
)], schema=schema_log)

# ------------------------------------------
# 6. Escrita no Delta Lake no container da camada Gold
# ------------------------------------------
log_execucao_df.write.format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .save("abfss://gold@dlsprojetofixo.dfs.core.windows.net/log_execucoes_gold_dim_cliente")

print(f"📌 Log da execução do job '{job_name}' registrado com sucesso.")


📌 Log da execução do job 'gold_dim_cliente' registrado com sucesso.


In [0]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS gold.log_execucoes_gold_dim_cliente
    USING DELTA
    LOCATION 'abfss://gold@dlsprojetofixo.dfs.core.windows.net/log_execucoes_gold_dim_cliente'
""")


DataFrame[]

In [0]:
%sql
SELECT * FROM gold.log_execucoes_gold_dim_cliente

job_name,data_execucao,qtd_linhas,status,erro,tempo_total_segundos
gold_dim_cliente,2025-04-22 02:18:15,0,ERRO,name 'dim_cliente_df' is not defined,0.0
gold_dim_cliente,2025-04-22 02:22:55,0,ERRO,name 'dim_cliente_df' is not defined,0.0
gold_dim_cliente,2025-04-22 02:23:21,0,ERRO,name 'dim_cliente_df' is not defined,0.0
gold_dim_cliente,2025-04-22 02:27:31,0,ERRO,name 'dim_cliente_df' is not defined,0.0
gold_dim_cliente,2025-04-22 02:30:46,500,SUCESSO,null,8.58
gold_dim_cliente,2025-04-23 00:17:21,500,SUCESSO,null,8.51
